<a href="https://colab.research.google.com/github/vvtrip/pyspark_course/blob/master/7%20-%20Case_Study_7_Perform_Data_Cleaning_and_Transformation_Op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  Load csv into spark as text file

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a6f7c2c38357ea4c658f9232dbfbed1e3fb6f606f87496341807d6baf16472de
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
raw = spark.read.text("./input.csv")

2. Parse the data as csv.

In [5]:
from pyspark.sql.types import Row

In [6]:
df = raw.rdd.map(lambda r:r[0].split(",")).map(lambda arr:Row(id=arr[1],app_name=arr[2],size_bytes=int(arr[3]),rating_count_tot=int(arr[6]),prime_genre=arr[12],screenshots=int(arr[14]),langnum=int(arr[15]))).toDF()

3. Convert bytes to MB and GB in a new column

In [7]:
from pyspark.sql.functions import col

In [8]:
df1 = df.withColumn("size_mb",df.size_bytes/1000000).withColumn("size_gb",col("size_mb")/1000)

4. List top 10 trending apps

In [9]:
df1.sort(df1.rating_count_tot.desc()).show(10)

+---------+--------------------+----------+----------------+-----------------+-----------+-------+----------+-------------------+
|       id|            app_name|size_bytes|rating_count_tot|      prime_genre|screenshots|langnum|   size_mb|            size_gb|
+---------+--------------------+----------+----------------+-----------------+-----------+-------+----------+-------------------+
|284882215|            Facebook| 389879808|         2974676|Social Networking|          1|     29|389.879808|0.38987980800000005|
|389801252|           Instagram| 113954816|         2161558|    Photo & Video|          0|     29|113.954816|        0.113954816|
|529479190|      Clash of Clans| 116476928|         2130805|            Games|          5|     18|116.476928|0.11647692800000001|
|420009108|          Temple Run|  65921024|         1724546|            Games|          5|      1| 65.921024|0.06592102400000001|
|284035177|Pandora - Music &...| 130242560|         1126879|            Music|          4|

5. The difference in the average number of screenshots displayed of highest and lowest rating apps.

In [10]:
import pyspark.sql.functions as F

In [11]:
min_rating, max_rating = df1.agg(F.min(df1.rating_count_tot),F.max(df1.rating_count_tot)).collect()[0]

In [12]:
df1.filter(df1.rating_count_tot == min_rating).agg(F.avg(df1.screenshots)).show()

+-----------------+
| avg(screenshots)|
+-----------------+
|2.572057205720572|
+-----------------+



In [13]:
df1.filter(df1.rating_count_tot == max_rating).agg(F.avg(df1.screenshots)).show()

+----------------+
|avg(screenshots)|
+----------------+
|             1.0|
+----------------+



6. What percentage of high rated apps support multiplelanguages

In [14]:
df1.filter(df1.langnum > 1).count() * 100 / df1.count()

46.448326055312954

7. How does app details contribute to user ratings?

In [15]:
percentiles = df1.stat.approxQuantile("rating_count_tot",[0.25,0.50,0.75],0.0)

In [16]:
df_25 = df1.filter(df1.rating_count_tot < percentiles[0])

In [17]:
df_50 = df1.filter((df1.rating_count_tot >= percentiles[0]) & (df.rating_count_tot < percentiles[1]))

In [18]:
df_75 = df1.filter((df1.rating_count_tot >= percentiles[1]) & (df.rating_count_tot < percentiles[2]))

In [19]:
df_100 = df1.filter(df1.rating_count_tot >= percentiles[2])

In [20]:
df_25.agg(F.avg(df_25.langnum)).show()

+-----------------+
|     avg(langnum)|
+-----------------+
|2.691948658109685|
+-----------------+



In [21]:
df_50.agg(F.avg(df_50.langnum)).show()

+-----------------+
|     avg(langnum)|
+-----------------+
|4.491860465116279|
+-----------------+



In [22]:
df_75.agg(F.avg(df_75.langnum)).show()

+-----------------+
|     avg(langnum)|
+-----------------+
|5.859720605355064|
+-----------------+



In [23]:
df_100.agg(F.avg(df_100.langnum)).show()

+----------------+
|    avg(langnum)|
+----------------+
|8.29511059371362|
+----------------+



8. Compare the statistics of different app groups/genres.

In [24]:
df1.groupBy("prime_genre").agg(F.avg(df1.langnum),F.avg(df1.screenshots),F.avg(df1.rating_count_tot),F.avg(df1.size_mb)).show()

+-----------------+------------------+------------------+---------------------+------------------+
|      prime_genre|      avg(langnum)|  avg(screenshots)|avg(rating_count_tot)|      avg(size_mb)|
+-----------------+------------------+------------------+---------------------+------------------+
|        Education| 7.235431235431236| 4.389277389277389|   1892.3076923076924|183.53252843589752|
|       Navigation| 5.380952380952381|2.5952380952380953|   4760.5952380952385|109.59350902380952|
|    Entertainment| 4.483234714003944|2.9940828402366866|    7694.905325443787|101.06431160946755|
|           Sports|  4.03030303030303|2.9696969696969697|    10273.89898989899| 79.33482408080808|
|     Food & Drink|3.7586206896551726| 2.086206896551724|   14796.034482758621| 78.15943062068966|
|    Photo & Video| 9.581168831168831| 2.987012987012987|   14258.363636363636| 67.13316405194799|
|           Travel| 7.552631578947368|2.5789473684210527|   11881.776315789473| 80.58766821052629|
|         

9. Does length of app description contribute to the ratings?

In [25]:
df1 = df1.withColumn("name_len", F.length(df1.app_name))

In [26]:
percentiles = df1.stat.approxQuantile("rating_count_tot",[0.25,0.50,0.75],0.0)

In [27]:
df_25 = df1.filter(df1.rating_count_tot < percentiles[0])

In [28]:
df_50 = df1.filter((df1.rating_count_tot >= percentiles[0]) & (df.rating_count_tot < percentiles[1]))

In [29]:
df_75 = df1.filter((df1.rating_count_tot >= percentiles[1]) & (df.rating_count_tot < percentiles[2]))

In [30]:
df_100 = df1.filter(df1.rating_count_tot >= percentiles[2])

In [32]:
df_25.agg(F.avg(df_25.name_len)).show()

+------------------+
|     avg(name_len)|
+------------------+
|23.085180863477245|
+------------------+



In [33]:
df_50.agg(F.avg(df_50.name_len)).show()

+-----------------+
|    avg(name_len)|
+-----------------+
|26.81046511627907|
+-----------------+



In [34]:
df_75.agg(F.avg(df_75.name_len)).show()

+-----------------+
|    avg(name_len)|
+-----------------+
|23.79743888242142|
+-----------------+



In [35]:
df_100.agg(F.avg(df_100.name_len)).show()

+------------------+
|     avg(name_len)|
+------------------+
|23.872526193247964|
+------------------+

